In [7]:
# load csv files in folder to map_data and fire_perim pd
import pandas as pd
import os
import glob
import numpy as np
import datetime
import matplotlib.pyplot as plt

# load FirePerimeters.csv to fire_perim pd dataframe
fire_perim = pd.read_csv('/Users/divya/Desktop/Project4/FirePerimeters.csv')

fire_perim

,OBJECTID,Year,State,Agency,Unit ID,Fire Name,Local Incident Number,Alarm Date,Containment Date,Cause,...,Management Objective,GIS Calculated Acres,Comments,Complex Name,IRWIN ID,Fire Number (historical use),Complex ID,DECADES,Shape__Area,Shape__Length
0,1,2023.0,CA,CDF,SKU,WHITWORTH,00004808,6/17/2023 12:00:00 AM,6/17/2023 12:00:00 AM,5.0,...,1.0,5.729125,NaN,NaN,{7985848C-0AC2-4BA4-8F0E-29F778652E61},NaN,NaN,2020.0,4.140784e+04,1247.166034
1,2,2023.0,CA,LRA,BTU,KAISER,00010225,6/2/2023 12:00:00 AM,6/2/2023 12:00:00 AM,5.0,...,1.0,13.602380,NaN,NaN,{43EBCC88-B3AC-48EB-8EF5-417FE0939CCF},NaN,NaN,2020.0,9.345588e+04,1285.514550
2,3,2023.0,CA,CDF,AEU,JACKSON,00017640,7/1/2023 12:00:00 AM,7/2/2023 12:00:00 AM,2.0,...,1.0,27.814460,NaN,NaN,{B64E1355-BF1D-441A-95D0-BC1FBB93483B},NaN,NaN,2020.0,1.830285e+05,2697.587429
3,4,2023.0,CA,CDF,AEU,CARBON,00018821,7/11/2023 12:00:00 AM,7/11/2023 12:00:00 AM,9.0,...,1.0,58.760230,NaN,NaN,{CB41DB0A-E4B1-489D-A4EA-738F2CD6DB3B},NaN,NaN,2020.0,3.882220e+05,2548.738779
4,5,2023.0,CA,CDF,AEU,LIBERTY,00018876,7/11/2023 12:00:00 AM,7/12/2023 12:00:00 AM,14.0,...,1.0,70.979000,NaN,NaN,{F83F70A4-07A7-40B8-BD51-10CCC1C30D63},NaN,NaN,2020.0,4.664560e+05,5106.936526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22256,22257,NaN,CA,CCO,MRN,UKNOWN,NaN,NaN,NaN,14.0,...,1.0,1494.277000,1917-34(Yr Not Report)MarinCo FireChief Garber...,NaN,NaN,NaN,NaN,NaN,9.768233e+06,11458.048312
22257,22258,NaN,CA,CCO,MRN,UKNOWN,NaN,NaN,NaN,14.0,...,1.0,465.534600,1917-34(Yr Not Report)MarinCo FireChief Garber...,NaN,NaN,NaN,NaN,NaN,3.048457e+06,6265.090923
22258,22259,NaN,CA,CCO,MRN,UKNOWN,NaN,NaN,NaN,14.0,...,1.0,2927.242000,1917-34(Yr Not Report)MarinCo FireChief Garber...,NaN,NaN,NaN,NaN,NaN,1.919632e+07,18325.306274
22259,22260,NaN,CA,CCO,MRN,UKNOWN,NaN,NaN,NaN,14.0,...,1.0,62.012680,1917-34(Yr Not Report)MarinCo FireChief Garber...,NaN,NaN,NaN,NaN,NaN,4.048420e+05,2344.122215


In [8]:
print(fire_perim.columns)

Index(['OBJECTID', 'Year', 'State', 'Agency', 'Unit ID', 'Fire Name',
       'Local Incident Number', 'Alarm Date', 'Containment Date', 'Cause',
       'Collection Method', 'Management Objective', 'GIS Calculated Acres',
       'Comments', 'Complex Name', 'IRWIN ID', 'Fire Number (historical use)',
       'Complex ID', 'DECADES', 'Shape__Area', 'Shape__Length'],
      dtype='object')


In [9]:
#in fire_perim: drop the following columns: Comments, Complex Name, Decades, Fire Number (historical use), Complex ID
fire_perim = fire_perim.drop(columns=['Comments', 'Complex Name', 'DECADES', 'Fire Number (historical use)', 'Complex ID', 'State'])


In [10]:
fire_perim.columns = fire_perim.columns.str.lower()
fire_perim.columns = fire_perim.columns.str.replace(' ', '_')
fire_perim.columns = fire_perim.columns.str.replace('__', '_')

fire_perim['year'] = fire_perim['year'].fillna(0).astype(int)
fire_perim['cause'] = fire_perim['cause'].fillna(0).astype(int)
fire_perim['collection_method'] = fire_perim['collection_method'].fillna(0).astype(int)
fire_perim['management_objective'] = fire_perim['management_objective'].fillna(0).astype(int)


In [11]:
fire_perim.dtypes

objectid                   int64
year                       int64
agency                    object
unit_id                   object
fire_name                 object
local_incident_number     object
alarm_date                object
containment_date          object
cause                      int64
collection_method          int64
management_objective       int64
gis_calculated_acres     float64
irwin_id                  object
shape_area               float64
shape_length             float64
dtype: object

In [12]:
#drop rows in which year = 0 
fire_perim = fire_perim[fire_perim['year'] != 0]
fire_perim

,objectid,year,agency,unit_id,fire_name,local_incident_number,alarm_date,containment_date,cause,collection_method,management_objective,gis_calculated_acres,irwin_id,shape_area,shape_length
0,1,2023,CDF,SKU,WHITWORTH,00004808,6/17/2023 12:00:00 AM,6/17/2023 12:00:00 AM,5,1,1,5.729125,{7985848C-0AC2-4BA4-8F0E-29F778652E61},4.140784e+04,1247.166034
1,2,2023,LRA,BTU,KAISER,00010225,6/2/2023 12:00:00 AM,6/2/2023 12:00:00 AM,5,1,1,13.602380,{43EBCC88-B3AC-48EB-8EF5-417FE0939CCF},9.345588e+04,1285.514550
2,3,2023,CDF,AEU,JACKSON,00017640,7/1/2023 12:00:00 AM,7/2/2023 12:00:00 AM,2,1,1,27.814460,{B64E1355-BF1D-441A-95D0-BC1FBB93483B},1.830285e+05,2697.587429
3,4,2023,CDF,AEU,CARBON,00018821,7/11/2023 12:00:00 AM,7/11/2023 12:00:00 AM,9,1,1,58.760230,{CB41DB0A-E4B1-489D-A4EA-738F2CD6DB3B},3.882220e+05,2548.738779
4,5,2023,CDF,AEU,LIBERTY,00018876,7/11/2023 12:00:00 AM,7/12/2023 12:00:00 AM,14,1,1,70.979000,{F83F70A4-07A7-40B8-BD51-10CCC1C30D63},4.664560e+05,5106.936526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22179,22180,1898,CCO,VNC,MATILIJA,00000000,4/1/1898 12:00:00 AM,NaN,14,0,1,2641.123000,NaN,1.577012e+07,27417.390907
22180,22181,1898,CCO,VNC,COZY DELL,00000000,9/1/1898 12:00:00 AM,NaN,14,0,1,2974.585000,NaN,1.775922e+07,18817.257350
22181,22182,1896,CCO,LAC,,00000000,NaN,NaN,14,0,1,7531.717000,NaN,4.473243e+07,36532.907435
22182,22183,1895,CCO,LAC,,00000000,NaN,NaN,14,0,1,628.113600,NaN,3.727269e+06,7182.120280


In [13]:
#print column names
print(fire_perim.columns)


Index(['objectid', 'year', 'agency', 'unit_id', 'fire_name',
       'local_incident_number', 'alarm_date', 'containment_date', 'cause',
       'collection_method', 'management_objective', 'gis_calculated_acres',
       'irwin_id', 'shape_area', 'shape_length'],
      dtype='object')


In [14]:
wims = pd.read_csv('/Users/divya/Desktop/Project4/WIMS_OBSERVATIONS.csv')

ParserError: Error tokenizing data. C error: Expected 45 fields in line 32325745, saw 47


In [ ]:
wims.head()